text

In [1]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
import os
import tarfile

def extract_tar_files(src_dir, dest_dir):
    # Create destination directory if it doesn't exist
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Iterate through files in source directory
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            if file.endswith('.tar.gz'):
                # Construct source and destination paths
                src_path = os.path.join(root, file)
                rel_path = os.path.relpath(src_path, src_dir)
                dest_path = os.path.join(dest_dir, rel_path[:-7])  # Remove '.tar.gz' extension

                # Create destination directory if it doesn't exist
                os.makedirs(dest_path, exist_ok=True)

                # Extract the tar.gz file
                print("Extracting:", src_path, "to", dest_path)
                with tarfile.open(src_path, 'r:gz') as tar:
                    tar.extractall(path=dest_path)
                print()  # Print a new line after extracting each tar file

# Specify source and destination directories
src_directory = 'drive/MyDrive/CULane'
dest_directory = 'data/culane'

# Extract tar.gz files recursively
extract_tar_files(src_directory, dest_directory)


Extracting: drive/MyDrive/CULane/list.tar.gz to data/culane/list

Extracting: drive/MyDrive/CULane/laneseg_label_w16.tar.gz to data/culane/laneseg_label_w16

Extracting: drive/MyDrive/CULane/annotations_new.tar.gz to data/culane/annotations_new

Extracting: drive/MyDrive/CULane/training_validation/driver_182_30frame.tar.gz to data/culane/training_validation/driver_182_30frame

Extracting: drive/MyDrive/CULane/training_validation/driver_161_90frame.tar.gz to data/culane/training_validation/driver_161_90frame

Extracting: drive/MyDrive/CULane/training_validation/driver_23_30frame.tar.gz to data/culane/training_validation/driver_23_30frame

Extracting: drive/MyDrive/CULane/testing/driver_193_90frame.tar.gz to data/culane/testing/driver_193_90frame

Extracting: drive/MyDrive/CULane/testing/driver_100_30frame.tar.gz to data/culane/testing/driver_100_30frame

Extracting: drive/MyDrive/CULane/testing/driver_37_30frame.tar.gz to data/culane/testing/driver_37_30frame



text

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error
import time
import cv2
import numpy as np

In [ ]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    # Convert nonzero pixels to (255, 255, 255)
    image[image != 0] = 255
    return image

def construct_dataframe(text_file_path, raw_image_dir, resulting_image_dir):
    data = []
    counter = 0
    with open(text_file_path, 'r') as file:
        for line in file:
            counter += 1
            if counter % 9999 != 0:
                continue
            # Split the line into components
            components = line.strip().split()
            raw_relative_path = components[0].strip()
            first_directory = raw_relative_path.split('/')[1]
            raw_image_path = raw_image_dir.strip() + '/' + first_directory + '/' + raw_relative_path
            resulting_image_path = resulting_image_dir.strip() + '/' + components[1].strip()
            bool_values = [bool(int(val)) for val in components[2:6]]

            # Load raw and resulting images
            raw_image = cv2.imread(raw_image_path)
            raw_image = cv2.resize(raw_image, None, fx=.1, fy=.1)
            resulting_image = cv2.imread(resulting_image_path)
            resulting_image[resulting_image != 0] = 255
            resulting_image = cv2.resize(resulting_image, None, fx=.1, fy=.1)

            # Append to data list
            data.append([raw_image, *bool_values, resulting_image])

    # Construct DataFrame
    df = pd.DataFrame(data, columns=['Raw_Image', 'Bool_1', 'Bool_2', 'Bool_3', 'Bool_4', 'Resulting_Image'])
    return df

In [ ]:
train_text_file_path = 'data/culane/list/list/train_gt.txt'
validation_text_file_path = 'data/culane/list/list/val_gt.txt'
raw_image_dir = 'data/culane/training_validation'
resulting_image_dir = 'data/culane/laneseg_label_w16'

train_df = construct_dataframe(train_text_file_path, raw_image_dir, resulting_image_dir)
validation_df = construct_dataframe(validation_text_file_path, raw_image_dir, resulting_image_dir)

In [ ]:
def standard_metrics(y_true, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate precision, recall, and F1 score
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(y_true, y_pred)

    return accuracy, precision, recall, f1, mse

In [ ]:
def flatten_split_dataframe(df):
    # Extract features from image columns and reshape into appropriate format
    X_image = df['Raw_Image']
    y = df['Resulting_Image']  # Flatten the last column (image)
    return X_image, y

batch_size = 8

def train(model, X_train, y_train, validation_df):
    X_batches = np.array_split(X_train, len(X_train) // batch_size)
    y_batches = np.array_split(y_train, len(y_train) // batch_size)
    for X_batch, y_batch in zip(X_batches, y_batches):
        # Fit the model to the training data
        model.fit(X_batches, y_batches)

        # Once the model is trained, you can use it to make predictions on new data
        # Assuming X_val is your validation features
        X_val, y_val = flatten_split_dataframe(validation_df)

        # Make predictions
        predictions = model.predict(X_val)

        # Evaluate predictions on validation set
        accuracy_val, precision_val, recall_val, f1_val, mse_val = standard_metrics(predictions, y_val)
        print("Linear Regression Metrics on Validation Set:")
        print("Accuracy:", accuracy_val)
        print("Precision:", precision_val)
        print("Recall:", recall_val)
        print("F1 Score:", f1_val)
        print("Mean Squared Error:", mse_val)

In [ ]:
# Assuming X_train and y_train are your training features and target respectively
X_train, y_train = flatten_split_dataframe(train_df)

# Create a Linear Regression model
model = LinearRegression()

train(model, X_train, y_train, validation_df)